DATA:
Sales (Training Data),
Transactions, &
Events

OUTPUT: EDA & Sales Forcasting using seasonaility


Import modules and setup dependacinesd

In [2]:
# BASE
# ------------------------------------------------------
import numpy as np
import pandas as pd
import os
import gc
import warnings
from pathlib import Path
#!pip install xgboost
import sklearn
from sklearn import linear_model
from sklearn.linear_model  import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, TimeSeriesSplit 
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score
pd.set_option('display.max_colwidth', None)
import statsmodels.api as sm

import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from statsmodels.tsa.statespace.sarimax import SARIMAX


from pandas import datetime
from pandas import read_csv
from pandas import DataFrame
from statsmodels.tsa.arima.model import ARIMA
from matplotlib import pyplot

# DATA VISUALIZATION
# ------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# CONFIGURATIONS
# ------------------------------------------------------
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format
warnings.filterwarnings('ignore')
folder = 'C:/Users/eflanag/OneDrive - FTI Consulting/Documents/Projects/Ibex_EF/IbexV2/ibexgit/kaggle-store-sales/data/'

C:\Users\eflanag\AppData\Local\Temp\ipykernel_24728\3927126861.py:27: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  from pandas import datetime


In [ ]:
# Define Gloabl Path Variables
# ------------------------------------------------------
current_directory = os.getcwd()
parent_directory = os.path.abspath(os.path.join(current_directory, os.pardir))

print("Parent directory:", parent_directory)

DATA_PATH = Path(parent_directory)
RAW_PATH = DATA_PATH / 'raw'
TRAIN_PATH = RAW_PATH / 'train.csv'
TEST_PATH = RAW_PATH / 'test.csv'
DATA_PATH



In [14]:
import importlib.util

# Specify the absolute path to the data_setup.py file
data_setup_path = '../src/data_setup.py'

# Load the module from the file
spec = importlib.util.spec_from_file_location('data_setup', data_setup_path)
data_setup = importlib.util.module_from_spec(spec)
spec.loader.exec_module(data_setup)

d = data_setup.get_data()
metrics_df = pd.DataFrame(columns=['Model', 'cols', 'MAE', 'MSE', 'RMSE','RMSLE', 'R2'])


Parent directory: c:\Users\eflanag\OneDrive - FTI Consulting\Documents\Projects\Ibex_EF\IbexV2\ibexgit\kaggle-store-sales
(3029400, 81)
complete


Modeling v0.1 - Sales forcasting using seasonailtiy

In [ ]:
# Resample to reduce data for dev
dFull = d
#d = dFull.sample(frac=0.1, replace=True, random_state=1)
d.shape

In [16]:
# preform test train split
cols = ['test/train' ,'sales', 'family', 'date', 'month','day_of_month', 'week_of_year','day_of_week', 'type', 'dcoilwtico_interpolated']

dataTrainTest = d[cols]

dataTrain = dataTrainTest[dataTrainTest['test/train'] ==  'train'].reset_index(drop=True)
dataTest = dataTrainTest[dataTrainTest['test/train'] ==  'test'].reset_index(drop=True)

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
dataTrain["family_encoded"] = encoder.fit_transform(dataTrain["family"])
dataTrain.drop('family', axis=1, inplace=True)


from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
dataTrain["type_encoded"] = encoder.fit_transform(dataTrain["type"])
dataTrain.drop('type', axis=1, inplace=True)

train_ = (dataTrain)
test_ = (dataTest)

train_, test_ = train_test_split(dataTrain.dropna().reset_index(drop=True), test_size=0.2, random_state=42)

del d
gc.collect()


8

In [ ]:
def predict_test(test_data, model):
    return model.predict(test_data)

model_metrics_full = []

X_train = 0
y_train = 0
X_test = 0
y_test = 0
y_pred = 0

def regressive_model(train_data, test_data, model, model_name):
    global metrics_df, X_train,y_train, X_test, y_test, y_pred
    model_metrics = {}

    X_train = train_data.drop(['sales','date', 'test/train'], axis=1) 
    y_train = train_data.sales 

    X_test = test_data.drop(['sales','date', 'test/train'], axis=1) 
    y_test = test_data.sales

    scaler = MinMaxScaler()

     # Run regression model
    mod = model
    mod.fit(X_train, y_train)
    X_train_ped =  mod.predict(X_train)
    X_train_ped[X_train_ped < 0] = 0
    X_train['y_pred'] = X_train_ped
    X_train['sales'] = y_train
    X_train['pred_diff'] = abs(X_train['sales'] - X_train['y_pred'])
    X_train['pred_diff_%'] = round(X_train['y_pred'] / X_train['sales'], 2)
    y_pred = mod.predict(X_test)
    print(X_train.shape)
    
    min_value = min(min(y_test), min(y_pred))
    y_test_transformed = y_test - min_value + 1
    y_pred_transformed = y_pred - min_value + 1

    y_pred[y_pred < 0] = 0

    X_test['y_pred'] = y_pred

    if (y_pred < 0).any():
        rmsle = 9999999

    else:    
        rmsle = np.sqrt(mean_squared_log_error(y_test, y_pred))

    # Evaluate the model
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    #rmsle = np.sqrt(mean_squared_log_error(y_test_transformed, y_pred_transformed))

    r2 = r2_score(y_test, y_pred)

    print('Columns:',list(X_train.columns))
    print(f'MAE: {mae:.2f}')
    print(f'MSE: {mse:.2f}')
    print(f'RMSE: {rmse:.2f}')
    print(f'RMSLE: {rmsle:.2f}')
    print(f'R2 score: {r2:.2f}')

    model_metrics[model_name] = {
        'Columns': (X_train.columns.values),
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'R2 score': r2
    }
    model_metrics_full.append(model_metrics) 
    metrics_df = metrics_df.append({'Model': model_name, 'cols': list(X_train.columns),  'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'RMSLE': rmsle, 'R2': r2}, ignore_index=True)

    return model

# Call model frame work for linear regression

In [ ]:
model = regressive_model(train_, test_, LinearRegression(),'LinearRegression')

In [ ]:
# Call model frame work for XGBoost
model = regressive_model(train_, test_, xgb.XGBRegressor(n_estimators=100,
                                           learning_rate=0.2), 
                                           'XGBoost')

In [ ]:
# Call model frame work for random forest regressor 
model = regressive_model(train_, test_, 
                 RandomForestRegressor(n_estimators=100,
                                       max_depth=10),        
                                       'RandomForest')

Next steps - Current leading RMSLE is 0.3

How can we improve our modeling? Utilise more features? Oil & Events. In general how does oil prices affect sales of differnent categories?

In [ ]:
# fit an ARIMA model and plot residual errors
from pandas import datetime
from pandas import read_csv
from pandas import DataFrame
from statsmodels.tsa.arima.model import ARIMA
from matplotlib import pyplot
# load dataset
def parser(x):
	return datetime.strptime('190'+x, '%Y-%m')


#series = read_csv('shampoo-sales.csv', header=0, index_col=0, parse_dates=True, squeeze=True, date_parser=parser)
train_
series = train_.copy()
series.set_index('date', inplace=True)
series
series.index = series.index.to_period('M')
# # fit model
series = series['sales']
model = ARIMA(series, order=(1,0,12))
model_fit = model.fit()
# summary of fit model
#print(model_fit.summary())




test_ARIMA = test_['sales'].copy()

#test_ARIMA.set_index('date', inplace=True)
#test_ARIMA.index = test_ARIMA.index.to_period('M')
#test_ARIMA['date'] = test_ARIMA.index

#test_ARIMA = test_ARIMA['sales']
test_ARIMA
#y_pred_ARIMA = model.predict(test_ARIMA)
#y_pred_ARIMA = model_fit.forecast(test_ARIMA['date'])
y_pred_ARIMA = model_fit.forecast(steps=len(test_.copy()['date']))


y_pred_ARIMA

y_pred_ARIMA[y_pred_ARIMA < 0] = 0

test_['y_pred'] = y_pred_ARIMA

if (y_pred_ARIMA < 0).any():
    rmsle = 9999999

else:    
    rmsle = np.sqrt(mean_squared_log_error(test_ARIMA, y_pred_ARIMA))

# Evaluate the model
mae = mean_absolute_error(test_ARIMA, y_pred_ARIMA)
mse = mean_squared_error(test_ARIMA, y_pred_ARIMA)
rmse = np.sqrt(mse)
#rmsle = np.sqrt(mean_squared_log_error(y_test_transformed, y_pred_transformed))

r2 = r2_score(test_ARIMA, y_pred_ARIMA)

print(f'MAE: {mae:.2f}')
print(f'MSE: {mse:.2f}')
print(f'RMSE: {rmse:.2f}')
print(f'RMSLE: {rmsle:.2f}')
print(f'R2 score: {r2:.2f}')

In [ ]:
dataTrain

In [28]:
dataTrain = dataTrain.sort_values(['family_encoded','date', 'type_encoded'])
dataTrain

for family in dataTrain['family_encoded'].unique():
    print(family)
    dataTrainFamily =  dataTrain[dataTrain['family_encoded'] == family ]

        
    # Find the last month in the 'date' column
    last_month = dataTrainFamily['date'].max().to_period('M')

    # Calculate the start date for the test data
    start_date = last_month.start_time + pd.DateOffset(days=1)

    # Filter the DataFrame based on the dynamic timeframe
    train_ = dataTrainFamily[dataTrainFamily['date'] < start_date]
    test_ = dataTrainFamily[dataTrainFamily['date'] >= start_date]
    print(train_.shape, test_.shape)

    series = train_.copy()
    #series = series.sort_values(['date','family_encoded'])
    series.set_index('date', inplace=True)
    series
    series.index = series.index.to_period('M')


    test_inner = test_.copy()
    #test_inner = test_inner.sort_values(['date','family_encoded'])
    test_inner.set_index('date', inplace=True)
    test_inner
    test_inner.index = test_inner.index.to_period('M')

    test_exog = test_inner.copy()
    test_exog = test_exog.drop(['test/train',  'sales'] , axis=1)
    exogenous = series.drop(['test/train',  'sales'] , axis=1)

    test_ARIMA = test_inner['sales']

    print('series', series.shape)
    print('exogenous', exogenous.shape)

    print('test_inner', test_inner.shape)
    print('test_exog', test_exog.shape)


    #(p, d, q, m)
    old_train = 0
    old_test = 0

    def fit_sarimax(data,exogenous):
        # Fit SARIMAX model to the subset of data
        so = (1, 0, 1, 12)
        model = SARIMAX(data, order=(1, 0, 4), exog=exogenous, seasonal_order=(so))
        model_fit = model.fit()
        return model_fit

    pdqm = [
    (1, 0, 1, 12)
    #,(2, 1, 2, 12),
    #(0, 1, 1, 7),
    #(1, 0, 0, 4),
    #(0, 0, 1, 24)
    ]

    for so in pdqm:
        #for i, (train_index, test_index) in enumerate(tscv.split(train_.copy())):
        print(f"pdqm {so}:")
        #print(f"  Train: index={train_index[0]}", f"  Train: index={train_index[-1]}")
        #print(f"  Test:  index={test_index[0]}", f"  Test:  index={test_index[-1]}")

        print('series', series.shape)
        print('test_exog', test_exog.shape)
        #exogenous = exogenous.iloc[0:5000]
        #series = series.iloc[0:5000]

        # # fit model
        #exogenous = series.copy()
        #series = series['sales']
        #exogenous = exogenous.drop(['test/train',  'sales'] , axis=1)
        #print(seasonal_order.)

        exogenous['dcoilwtico_interpolated'] = exogenous['dcoilwtico_interpolated'].astype('int8')
        # Convert categorical columns to categorical data type
        exogenous['family_encoded'] = pd.Categorical(exogenous['family_encoded'])
        exogenous['type_encoded'] = pd.Categorical(exogenous['type_encoded'])
        exogenous['month'] = pd.Categorical(exogenous['month'])
        exogenous['day_of_month'] = pd.Categorical(exogenous['day_of_month'])
        exogenous['week_of_year'] = pd.Categorical(exogenous['week_of_year'])
        exogenous['day_of_week'] = pd.Categorical(exogenous['day_of_week'])

        series['sales'] = pd.to_numeric(series['sales'], downcast='integer')
        test_ARIMA = pd.to_numeric(test_ARIMA, downcast='integer')

        test_exog['dcoilwtico_interpolated'] = test_exog['dcoilwtico_interpolated'].astype('int8')
        # Convert categorical columns to categorical data type
        test_exog['family_encoded'] = pd.Categorical(test_exog['family_encoded'])
        test_exog['type_encoded'] = pd.Categorical(test_exog['type_encoded'])
        test_exog['month'] = pd.Categorical(test_exog['month'])
        test_exog['day_of_month'] = pd.Categorical(test_exog['day_of_month'])
        test_exog['week_of_year'] = pd.Categorical(test_exog['week_of_year'])
        test_exog['day_of_week'] = pd.Categorical(test_exog['day_of_week'])        
    

        #data_types = exogenous.dtypes
        #print("Data Types:")
        #print(data_types)

        # Get the memory usage of each column
        memory_usage = exogenous.memory_usage(deep=True)
        print("\nMemory Usage:")
        print(memory_usage)
        
        # Calculate memory usage in bytes
        memory_bytes = exogenous.memory_usage(deep=True).sum()
        # Convert to gigabytes (GB)
        memory_gb = memory_bytes / (1024**3)
        # Print the size in GB
        #print(f"Size of data: {memory_gb:.2f} GB")

        start_time = time.time()
        #model = ARIMA(series, order=(1,0,4), exog=exogenous )



        model = SARIMAX(series['sales'], order=(1, 0, 4), exog=exogenous, seasonal_order=(so))
        model_fit = model.fit()
        del model



        y_pred_ARIMA = model_fit.forecast(steps=len(test_inner['sales']), exog= test_exog)

        y_pred_ARIMA[y_pred_ARIMA < 0] = 0

        #test_['y_pred'] = y_pred_ARIMA

        if (y_pred_ARIMA < 0).any():
            rmsle = 9999999

        else:    
            print((test_ARIMA.shape),(y_pred_ARIMA.shape))
            print(type(test_ARIMA),type(y_pred_ARIMA.shape))

            rmsle = np.sqrt(mean_squared_log_error(test_ARIMA, y_pred_ARIMA))

        # Evaluate the model
        mae = mean_absolute_error(test_ARIMA, y_pred_ARIMA)
        mse = mean_squared_error(test_ARIMA, y_pred_ARIMA)
        rmse = np.sqrt(mse)
        #rmsle = np.sqrt(mean_squared_log_error(y_test_transformed, y_pred_transformed))

        r2 = r2_score(test_ARIMA, y_pred_ARIMA)

        print(f'MAE: {mae:.2f}')
        print(f'MSE: {mse:.2f}')
        print(f'RMSE: {rmse:.2f}')
        print(f'RMSLE: {rmsle:.2f}')
        print(f'R2 score: {r2:.2f}')

        #del 
        
        metrics_df = metrics_df.append({'Model': 'SARIMAX', 'cols': list(train_.columns),  'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'RMSLE': rmsle, 'R2': r2}, ignore_index=True)
        print('rolling mean RMSLE',metrics_df['RMSLE'].mean())
        del y_pred_ARIMA, test_exog, series, exogenous
        gc.collect()
        
        #print(dataTrainFamily)




0
(90180, 10) (756, 10)
series (90180, 9)
exogenous (90180, 7)
test_inner (756, 9)
test_exog (756, 7)
pdqm (1, 0, 1, 12):
series (90180, 9)
test_exog (756, 7)

Memory Usage:
Index                      721440
month                       90576
day_of_month                91500
week_of_year                92708
day_of_week                 90536
dcoilwtico_interpolated     90180
family_encoded              90296
type_encoded                90392
dtype: int64
(756,) (756,)
<class 'pandas.core.series.Series'> <class 'tuple'>
MAE: 3.82
MSE: 23.85
RMSE: 4.88
RMSLE: 0.70
R2 score: 0.22
rolling mean RMSLE 1.4658753602011774
1
(90180, 10) (756, 10)
series (90180, 9)
exogenous (90180, 7)
test_inner (756, 9)
test_exog (756, 7)
pdqm (1, 0, 1, 12):
series (90180, 9)
test_exog (756, 7)

Memory Usage:
Index                      721440
month                       90576
day_of_month                91500
week_of_year                92708
day_of_week                 90536
dcoilwtico_interpolated     90180


In [29]:
metrics_df.sort_values(by='RMSLE')

,Model,cols,MAE,MSE,RMSE,RMSLE,R2
4,SARIMAX,"[test/train, sales, date, month, day_of_month, week_of_year, day_of_week, dcoilwtico_interpolated, family_encoded, type_encoded]",0.01,0.01,0.12,0.07,-0.01
17,SARIMAX,"[test/train, sales, date, month, day_of_month, week_of_year, day_of_week, dcoilwtico_interpolated, family_encoded, type_encoded]",0.01,0.01,0.12,0.07,-0.01
26,SARIMAX,"[test/train, sales, date, month, day_of_month, week_of_year, day_of_week, dcoilwtico_interpolated, family_encoded, type_encoded]",0.13,0.03,0.17,0.14,-1.25
23,SARIMAX,"[test/train, sales, date, month, day_of_month, week_of_year, day_of_week, dcoilwtico_interpolated, family_encoded, type_encoded]",0.32,0.32,0.56,0.30,-0.00
1,SARIMAX,"[test/train, sales, date, month, day_of_month, week_of_year, day_of_week, dcoilwtico_interpolated, family_encoded, type_encoded]",0.17,0.35,0.59,0.31,-0.10
14,SARIMAX,"[test/train, sales, date, month, day_of_month, week_of_year, day_of_week, dcoilwtico_interpolated, family_encoded, type_encoded]",0.17,0.35,0.59,0.31,-0.10
39,SARIMAX,"[test/train, sales, date, month, day_of_month, week_of_year, day_of_week, dcoilwtico_interpolated, family_encoded, type_encoded]",0.47,0.38,0.62,0.40,-0.32
34,SARIMAX,"[test/train, sales, date, month, day_of_month, week_of_year, day_of_week, dcoilwtico_interpolated, family_encoded, type_encoded]",1565.59,4006158.19,2001.54,0.47,0.34
29,SARIMAX,"[test/train, sales, date, month, day_of_month, week_of_year, day_of_week, dcoilwtico_interpolated, family_encoded, type_encoded]",390.58,272803.02,522.31,0.49,0.33
40,SARIMAX,"[test/train, sales, date, month, day_of_month, week_of_year, day_of_week, dcoilwtico_interpolated, family_encoded, type_encoded]",102.01,18965.49,137.72,0.49,0.29


In [32]:
data_1 = dataTrain[dataTrain['family_encoded'] == 12 ]
data_1.to_csv('family1.csv')

In [18]:
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from statsmodels.tsa.statespace.sarimax import SARIMAX
import time

import multiprocessing as mp


#tscv = TimeSeriesSplit(2, test_size=250)
#print(tscv)

# Find the last month in the 'date' column
last_month = dataTrain['date'].max().to_period('M')

# Calculate the start date for the test data
start_date = last_month.start_time + pd.DateOffset(days=1)

# Filter the DataFrame based on the dynamic timeframe
train_ = dataTrain[dataTrain['date'] < start_date]
test_ = dataTrain[dataTrain['date'] >= start_date]

series = train_.copy()
series = series.sort_values(['date','family_encoded'])
series.set_index('date', inplace=True)
series
#series.index = series.index.to_period('M')


test_inner = test_.copy()
test_inner = test_inner.sort_values(['date','family_encoded'])
test_inner.set_index('date', inplace=True)
test_inner
#test_inner.index = test_inner.index.to_period('M')

test_exog = test_inner.copy()
test_exog = test_exog.drop(['test/train',  'sales'] , axis=1)

print('test_inner', test_inner.shape)
print('test_exog', test_exog.shape)

test_ARIMA = test_inner['sales']

print('test_inner', test_inner.shape)
print('test_exog', test_exog.shape)

exogenous = series.drop(['test/train',  'sales'] , axis=1)

#(p, d, q, m)
old_train = 0
old_test = 0

def fit_sarimax(data,exogenous):
    # Fit SARIMAX model to the subset of data
    so = (1, 0, 1, 12)
    model = SARIMAX(data, order=(1, 0, 4), exog=exogenous, seasonal_order=(so))
    model_fit = model.fit()
    return model_fit

pdqm = [
(1, 0, 1, 12)
#,(2, 1, 2, 12),
#(0, 1, 1, 7),
#(1, 0, 0, 4),
#(0, 0, 1, 24)
]

for so in pdqm:
    #for i, (train_index, test_index) in enumerate(tscv.split(train_.copy())):
    print(f"pdqm {so}:")
    #print(f"  Train: index={train_index[0]}", f"  Train: index={train_index[-1]}")
    #print(f"  Test:  index={test_index[0]}", f"  Test:  index={test_index[-1]}")

    print('series', series.shape)
    print('test_exog', test_exog.shape)

    # # fit model
    #exogenous = series.copy()
    #series = series['sales']
    #exogenous = exogenous.drop(['test/train',  'sales'] , axis=1)
    #print(seasonal_order.)

    exogenous['dcoilwtico_interpolated'] = exogenous['dcoilwtico_interpolated'].astype('int8')
    # Convert categorical columns to categorical data type
    exogenous['family_encoded'] = pd.Categorical(exogenous['family_encoded'])
    exogenous['type_encoded'] = pd.Categorical(exogenous['type_encoded'])
    exogenous['month'] = pd.Categorical(exogenous['month'])
    exogenous['day_of_month'] = pd.Categorical(exogenous['day_of_month'])
    exogenous['week_of_year'] = pd.Categorical(exogenous['week_of_year'])
    exogenous['day_of_week'] = pd.Categorical(exogenous['day_of_week'])

    series['sales'] = pd.to_numeric(series['sales'], downcast='integer')
    test_ARIMA = pd.to_numeric(series['sales'], downcast='integer')
    

    data_types = exogenous.dtypes
    print("Data Types:")
    print(data_types)

    # Get the memory usage of each column
    memory_usage = exogenous.memory_usage(deep=True)
    print("\nMemory Usage:")
    print(memory_usage)
    
    # Calculate memory usage in bytes
    memory_bytes = exogenous.memory_usage(deep=True).sum()
    # Convert to gigabytes (GB)
    memory_gb = memory_bytes / (1024**3)
    # Print the size in GB
    print(f"Size of data: {memory_gb:.2f} GB")

    print(exogenous.head())
    start_time = time.time()
    #model = ARIMA(series, order=(1,0,4), exog=exogenous )

    num_processes = 8
    (data_chunks) = np.array_split(series['sales'], num_processes)
    (d2) = np.array_split(exogenous, num_processes)

    #data_chunks2 = np.array_split(exogenous, num_processes)
    #data_chunks = [(chunk, exogenous) for chunk in data_chunks]
    #data_chunks = np.array_split(data_chunks, num_processes)

    model = SARIMAX(series['sales'], order=(1, 0, 4), exog=exogenous, seasonal_order=(so))
    model_fit = model.fit()
    del model

    '''
    # Calculate the training time
    end_time = time.time()
    training_time = end_time - start_time
    print(f"Training time: {str(int(training_time))} seconds")

    pool = mp.Pool(processes=num_processes)
    results = pool.map(fit_sarimax,  (data_chunks,d2))


    fitted_models = []
    for result in results:
        fitted_models.append(result)
    '''

    





    y_pred_ARIMA = model_fit.forecast(steps=len(test_inner['sales']), exog= test_exog)

    y_pred_ARIMA[y_pred_ARIMA < 0] = 0

    #test_['y_pred'] = y_pred_ARIMA

    if (y_pred_ARIMA < 0).any():
        rmsle = 9999999

    else:    
        #print((test_ARIMA.shape),(y_pred_ARIMA.shape))
        #print(type(test_ARIMA),type(y_pred_ARIMA.shape))

        rmsle = np.sqrt(mean_squared_log_error(test_ARIMA, y_pred_ARIMA))

    # Evaluate the model
    mae = mean_absolute_error(test_ARIMA, y_pred_ARIMA)
    mse = mean_squared_error(test_ARIMA, y_pred_ARIMA)
    rmse = np.sqrt(mse)
    #rmsle = np.sqrt(mean_squared_log_error(y_test_transformed, y_pred_transformed))

    r2 = r2_score(test_ARIMA, y_pred_ARIMA)

    print(f'MAE: {mae:.2f}')
    print(f'MSE: {mse:.2f}')
    print(f'RMSE: {rmse:.2f}')
    print(f'RMSLE: {rmsle:.2f}')
    print(f'R2 score: {r2:.2f}')

    #del 
    

    metrics_df = metrics_df.append({'Model': 'SARIMAX', 'cols': list(train_.columns),  'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'RMSLE': rmsle, 'R2': r2}, ignore_index=True)


test_inner (24948, 9)
test_exog (24948, 7)
test_inner (24948, 9)
test_exog (24948, 7)
pdqm (1, 0, 1, 12):
series (2975940, 9)
test_exog (24948, 7)
Data Types:
month                      category
day_of_month               category
week_of_year               category
day_of_week                category
dcoilwtico_interpolated        int8
family_encoded             category
type_encoded               category
dtype: object

Memory Usage:
Index                      23807520
month                       2976336
day_of_month                2977260
week_of_year                2978468
day_of_week                 2976296
dcoilwtico_interpolated     2975940
family_encoded              2977276
type_encoded                2976152
dtype: int64
Size of data: 0.04 GB
           month day_of_month week_of_year day_of_week  \
date                                                     
2013-01-01     1            1            1           2   
2013-01-01     1            1            1           2   
2013-

KeyboardInterrupt: 

In [ ]:
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from statsmodels.tsa.statespace.sarimax import SARIMAX
import time


#tscv = TimeSeriesSplit(2, test_size=250)
#print(tscv)

# Find the last month in the 'date' column
last_month = dataTrain['date'].max().to_period('M')

# Calculate the start date for the test data
start_date = last_month.start_time + pd.DateOffset(days=1)

# Filter the DataFrame based on the dynamic timeframe
train_ = dataTrain[dataTrain['date'] < start_date]
test_ = dataTrain[dataTrain['date'] >= start_date]

series = train_.copy()
series.set_index('date', inplace=True)
series
series.index = series.index.to_period('M')


test_inner = test_.copy()
test_inner.set_index('date', inplace=True)
test_inner
test_inner.index = test_inner.index.to_period('M')

test_exog = test_inner.copy()
test_exog = test_exog.drop(['test/train',  'sales'] , axis=1)

print('test_inner', test_inner.shape)
print('test_exog', test_exog.shape)

test_ARIMA = test_inner['sales']

print('test_inner', test_inner.shape)
print('test_exog', test_exog.shape)

exogenous = series.drop(['test/train',  'sales'] , axis=1)

#(p, d, q, m)
old_train = 0
old_test = 0


scaler = MinMaxScaler()
data['previous_sales'] = scaler.fit_transform(data['previous_sales'].values.reshape(-1, 1))

# Split data into features (X) and target (y)
X = data[['category', 'location', 'holiday', 'previous_sales']].values
y = data['sales'].values

# Step 2: Splitting the Data
# Assuming you have already split your data into train and validation sets

# Split the data into train and validation sets
train_size = int(0.8 * len(X))
X_train, X_val = X[:train_size], X[train_size:]
y_train, y_val = y[:train_size], y[train_size:]

# Step 3: Create Input Sequences
# Assuming you have already created input sequences for your LSTM model

# Create input sequences with a specific time window
time_window = 10

def create_sequences(X, y, time_window):
    X_seq, y_seq = [], []
    for i in range(len(X) - time_window):
        X_seq.append(X[i : i+time_window])
        y_seq.append(y[i + time_window])
    return np.array(X_seq), np.array(y_seq)

X_train_seq, y_train_seq = create_sequences(X_train, y_train, time_window)
X_val_seq, y_val_seq = create_sequences(X_val, y_val, time_window)

# Step 4: Design and Train the LSTM Model

# Design the LSTM model
model = Sequential()
model.add(LSTM(64, input_shape=(time_window, X_train_seq.shape[2])))
model.add(Dense(1))

# Compile and train the model
model.compile(loss='mse', optimizer='adam')
model.fit(X_train_seq, y_train_seq, epochs=10, batch_size=32, validation_data=(X_val_seq, y_val_seq))

# Step 5: Evaluate and Fine-Tune the Model
# Assuming you have already evaluated and fine-tuned your model

# Evaluation on validation set
val_loss = model.evaluate(X_val_seq, y_val_seq)
print("Validation Loss:", val_loss)

# Step 6: Make Predictions
# Assuming you have new data for which you want to make predictions

# Assuming 'new_data' contains the new data with the same columns as the training data
new_data['category'] = label_encoder.transform(new_data['category'])
new_data['location'] = label_encoder.transform(new_data['location'])
new_data['holiday'] = label_encoder.transform(new_data['holiday'])
new_data['previous_sales'] = scaler.transform(new_data['previous_sales'].values.reshape(-1, 1))

# Create input sequence for prediction
X_pred_seq, _ = create_sequences(new_data[['category', 'location', 'holiday', 'previous_sales']].values, [], time_window)

# Make predictions
predictions = model.predict(X_pred_seq)

# Print the predictions
print("Predictions:", predictions)



for so in pdqm:
    #for i, (train_index, test_index) in enumerate(tscv.split(train_.copy())):
    print(f"pdqm {so}:")
    #print(f"  Train: index={train_index[0]}", f"  Train: index={train_index[-1]}")
    #print(f"  Test:  index={test_index[0]}", f"  Test:  index={test_index[-1]}")

    print('series', series.shape)
    print('test_exog', test_exog.shape)

    # # fit model
    #exogenous = series.copy()
    #series = series['sales']
    #exogenous = exogenous.drop(['test/train',  'sales'] , axis=1)

    start_time = time.time()
    #model = ARIMA(series, order=(1,0,4), exog=exogenous )
    model = SARIMAX(series['sales'], order=(1, 0, 4), exog=exogenous, seasonal_order=(so))
    model_fit = model.fit()

    # Calculate the training time
    end_time = time.time()
    training_time = end_time - start_time
    print(f"Training time: {str(int(training_time))} seconds")




    y_pred_ARIMA = model_fit.forecast(steps=len(test_inner['sales']), exog= test_exog)

    y_pred_ARIMA[y_pred_ARIMA < 0] = 0

    #test_['y_pred'] = y_pred_ARIMA

    if (y_pred_ARIMA < 0).any():
        rmsle = 9999999

    else:    
        #print((test_ARIMA.shape),(y_pred_ARIMA.shape))
        #print(type(test_ARIMA),type(y_pred_ARIMA.shape))

        rmsle = np.sqrt(mean_squared_log_error(test_ARIMA, y_pred_ARIMA))

    # Evaluate the model
    mae = mean_absolute_error(test_ARIMA, y_pred_ARIMA)
    mse = mean_squared_error(test_ARIMA, y_pred_ARIMA)
    rmse = np.sqrt(mse)
    #rmsle = np.sqrt(mean_squared_log_error(y_test_transformed, y_pred_transformed))

    r2 = r2_score(test_ARIMA, y_pred_ARIMA)

    print(f'MAE: {mae:.2f}')
    print(f'MSE: {mse:.2f}')
    print(f'RMSE: {rmse:.2f}')
    print(f'RMSLE: {rmsle:.2f}')
    print(f'R2 score: {r2:.2f}')

    #del 
    

    metrics_df = metrics_df.append({'Model': 'SARIMAX', 'cols': list(train_loop.columns),  'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'RMSLE': rmsle, 'R2': r2}, ignore_index=True)


In [ ]:
metrics_df.sort_values(by='RMSLE')

In [ ]:
import numpy as np
from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(2, test_size=250)
print(tscv)

train_loop = train_.copy()
test_loop = test.copy()

old_train = 0
old_test = 0
for i, (train_index, test_index) in enumerate(tscv.split(train_.copy())):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index[0]}", f"  Train: index={train_index[-1]}")
    print(f"  Test:  index={test_index[0]}", f"  Test:  index={test_index[-1]}")

    series = train_loop.iloc[train_index[0]:train_index[-1]].copy()
    series.shape
    series.set_index('date', inplace=True)
    series
    series.index = series.index.to_period('M')
    # # fit model
    exogenous = series.copy()
    series = series['sales']
    exogenous = exogenous.drop(['test/train',  'sales'] , axis=1)


    model = ARIMA(series, order=(1,0,4), exog=exogenous )
    model_fit = model.fit()
    # summary of fit model
    #print(model_fit.summary())

    test_inner = train_loop.iloc[test_index[0]:test_index[-1]].copy()
    test_inner.set_index('date', inplace=True)
    test_inner
    test_inner.index = test_inner.index.to_period('M')

    test_exog = test_inner.copy()
    test_exog = test_exog.drop(['test/train',  'sales'] , axis=1)

    print('test_inner', test_inner.shape)
    print('test_exog', test_exog.shape)

    test_ARIMA = test_inner['sales']

    print('test_inner', test_inner.shape)
    print('test_exog', test_exog.shape)

    y_pred_ARIMA = model_fit.forecast(steps=len(test_ARIMA.copy()), exog= test_exog)

    y_pred_ARIMA[y_pred_ARIMA < 0] = 0

    #test_['y_pred'] = y_pred_ARIMA

    if (y_pred_ARIMA < 0).any():
        rmsle = 9999999

    else:    
        #print((test_ARIMA.shape),(y_pred_ARIMA.shape))
        #print(type(test_ARIMA),type(y_pred_ARIMA.shape))

        rmsle = np.sqrt(mean_squared_log_error(test_ARIMA, y_pred_ARIMA))

    # Evaluate the model
    mae = mean_absolute_error(test_ARIMA, y_pred_ARIMA)
    mse = mean_squared_error(test_ARIMA, y_pred_ARIMA)
    rmse = np.sqrt(mse)
    #rmsle = np.sqrt(mean_squared_log_error(y_test_transformed, y_pred_transformed))

    r2 = r2_score(test_ARIMA, y_pred_ARIMA)

    print(f'MAE: {mae:.2f}')
    print(f'MSE: {mse:.2f}')
    print(f'RMSE: {rmse:.2f}')
    print(f'RMSLE: {rmsle:.2f}')
    print(f'R2 score: {r2:.2f}')
    old_test = test_index


In [ ]:
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from statsmodels.tsa.statespace.sarimax import SARIMAX

tscv = TimeSeriesSplit(2, test_size=250)
print(tscv)

train_loop = train_.copy()
test_loop = test.copy()
#(p, d, q, m)
old_train = 0
old_test = 0

pdqm = [(1, 0, 1, 12),
(2, 1, 2, 12),
(0, 1, 1, 7),
(1, 1, 0, 4),
(0, 0, 1, 24)]

for so in pdqm:
    for i, (train_index, test_index) in enumerate(tscv.split(train_.copy())):
        print(f"Fold {i}:")
        print(f"  Train: index={train_index[0]}", f"  Train: index={train_index[-1]}")
        print(f"  Test:  index={test_index[0]}", f"  Test:  index={test_index[-1]}")

        series = train_loop.iloc[train_index[0]:train_index[-1]].copy()
        series.shape
        series.set_index('date', inplace=True)
        series
        series.index = series.index.to_period('M')
        # # fit model
        exogenous = series.copy()
        #series = series['sales']
        #exogenous = exogenous.drop(['test/train',  'sales'] , axis=1)


        #model = ARIMA(series, order=(1,0,4), exog=exogenous )
        model = SARIMAX(series['sales'], order=(1, 0, 4), exog=series.drop(['test/train',  'sales'] , axis=1), seasonal_order=(so))

        model_fit = model.fit()
        # summary of fit model
        #print(model_fit.summary())

        test_inner = train_loop.iloc[test_index[0]:test_index[-1]].copy()
        test_inner.set_index('date', inplace=True)
        test_inner
        test_inner.index = test_inner.index.to_period('M')

        test_exog = test_inner.copy()
        test_exog = test_exog.drop(['test/train',  'sales'] , axis=1)

        print('test_inner', test_inner.shape)
        print('test_exog', test_exog.shape)

        test_ARIMA = test_inner['sales']

        print('test_inner', test_inner.shape)
        print('test_exog', test_exog.shape)

        y_pred_ARIMA = model_fit.forecast(steps=len(test_ARIMA.copy()), exog= test_exog)

        y_pred_ARIMA[y_pred_ARIMA < 0] = 0

        #test_['y_pred'] = y_pred_ARIMA

        if (y_pred_ARIMA < 0).any():
            rmsle = 9999999

        else:    
            #print((test_ARIMA.shape),(y_pred_ARIMA.shape))
            #print(type(test_ARIMA),type(y_pred_ARIMA.shape))

            rmsle = np.sqrt(mean_squared_log_error(test_ARIMA, y_pred_ARIMA))

        # Evaluate the model
        mae = mean_absolute_error(test_ARIMA, y_pred_ARIMA)
        mse = mean_squared_error(test_ARIMA, y_pred_ARIMA)
        rmse = np.sqrt(mse)
        #rmsle = np.sqrt(mean_squared_log_error(y_test_transformed, y_pred_transformed))

        r2 = r2_score(test_ARIMA, y_pred_ARIMA)

        print(f'MAE: {mae:.2f}')
        print(f'MSE: {mse:.2f}')
        print(f'RMSE: {rmse:.2f}')
        print(f'RMSLE: {rmsle:.2f}')
        print(f'R2 score: {r2:.2f}')

        #del 
        

        metrics_df = metrics_df.append({'Model': 'SARIMAX', 'cols': list(train_loop.columns),  'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'RMSLE': rmsle, 'R2': r2}, ignore_index=True)
